In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import os
project_folder = '/gdrive/MyDrive/ProjectCIRI'
my_module_path = os.path.join(project_folder, 'code', 'ciri_utils')

In [3]:
!pip install $my_module_path

Processing /gdrive/MyDrive/ProjectCIRI/code/ciri_utils
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 726.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 8.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-

In [4]:
from ciri_utils.engine_v2 import CIRI_trainer
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.train.torch import TorchTrainer
from ray.tune import Trainable

In [ ]:
selected_model="efficientnet_b0"
root_base = os.path.join(project_folder, 'Incidents-subset')
root_augmented = os.path.join(project_folder, 'augmented_images')

data_folders=[root_base, root_augmented]

In [ ]:
ciri_trainer = CIRI_trainer(model=selected_model,
                            data_folders=data_folders,
                            data_prop=0.8,
                            sample_indices=list(range(1000)), # remove this for full dataset
                            )

In [ ]:
param_space = {
    "epochs": 1,
    "batch_size": tune.grid_search([16, 32, 64]),
    "lr": tune.loguniform(1e-4, 1e-2)
}
ciri_trainable = ciri_trainer.get_trainer(run_name="test_tuning",
                                     as_trainable=True)

In [ ]:
results = ciri_trainer.tune_hyperparams(
	run_name="tune_test",
	param_space=param_space,
	num_samples=5
)

Defaulting to ASHA scheduler (no scheduler provided or not an instance of TrialScheduler)


/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-04-11 09:46:27,852	INFO worker.py:1752 -- Started a local Ray instance.
2024-04-11 09:46:30,315	INFO tune.py:263 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2024-04-11 09:46:30,326	INFO tune.py:622 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


+---------------------------------------------------------------------+
| Configuration for experiment     TorchTrainer_2024-04-11_09-46-23   |
+---------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator              |
| Scheduler                        AsyncHyperBandScheduler            |
| Number of trials                 15                                 |
+---------------------------------------------------------------------+

View detailed results here: /root/ray_results/TorchTrainer_2024-04-11_09-46-23
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-04-11_09-46-23_717788_319/artifacts/2024-04-11_09-46-30/TorchTrainer_2024-04-11_09-46-23/driver_artifacts`

Trial status: 15 PENDING
Current time: 2024-04-11 09:46:31. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+---------------------------------------------------------------------------------------+

(TrainTrainable pid=1908) 2024-04-11 09:46:49.783452: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(TrainTrainable pid=1908) 2024-04-11 09:46:49.783541: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(TrainTrainable pid=1908) 2024-04-11 09:46:49.785607: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(TrainTrainable pid=1908) 2024-04-11 09:46:52.018864: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



Trial TorchTrainer_5f9ce_00000 started with configuration:
+-------------------------------------------------------+
| Trial TorchTrainer_5f9ce_00000 config                 |
+-------------------------------------------------------+
| train_loop_config/batch_size                       16 |
| train_loop_config/epochs                            1 |
| train_loop_config/lr                      0.000275353 |
+-------------------------------------------------------+


...

Trial status: 14 TERMINATED | 1 RUNNING
Current time: 2024-04-11 11:01:39. Total running time: 1hr 15min 9s
Logical resource usage: 2.0/2 CPUs, 0/0 GPUs
+---------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                 status         train_loop_config/lr     ...config/batch_size     iter     total time (s)       loss     accuracy |
+---------------------------------------------------------------------------------------------------------------------------------------------+
| TorchTrainer_5f9ce_00014   RUNNING                 0.00073538                        64                                                     |
| TorchTrainer_5f9ce_00000   TERMINATED              0.000275353                       16        1            428.658   2.15022          0    |
| TorchTrainer_5f9ce_00001   TERMINATED              0.000963128                       32        1            293.741   1.11601

Epoch (training) 1:  88%|████████▊ | 23/26 [03:50<00:29,  9.84s/it] [repeated 2x across cluster]


Trial status: 14 TERMINATED | 1 RUNNING
Current time: 2024-04-11 11:02:10. Total running time: 1hr 15min 39s
Logical resource usage: 2.0/2 CPUs, 0/0 GPUs
+---------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                 status         train_loop_config/lr     ...config/batch_size     iter     total time (s)       loss     accuracy |
+---------------------------------------------------------------------------------------------------------------------------------------------+
| TorchTrainer_5f9ce_00014   RUNNING                 0.00073538                        64                                                     |
| TorchTrainer_5f9ce_00000   TERMINATED              0.000275353                       16        1            428.658   2.15022          0    |
| TorchTrainer_5f9ce_00001   TERMINATED              0.000963128                       32        1            293.741   1.1160

Epoch (test) 1: 100%|██████████| 7/7 [00:15<00:00,  2.22s/it]
(RayTrainWorker pid=21998) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/TorchTrainer_2024-04-11_09-46-23/TorchTrainer_5f9ce_00014_14_batch_size=64,lr=0.0007_2024-04-11_09-46-31/checkpoint_000000)
2024-04-11 11:02:39,940	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/TorchTrainer_2024-04-11_09-46-23' in 0.0483s.



Trial TorchTrainer_5f9ce_00014 completed after 1 iterations at 2024-04-11 11:02:39. Total running time: 1hr 16min 9s
+--------------------------------------------------------------+
| Trial TorchTrainer_5f9ce_00014 result                        |
+--------------------------------------------------------------+
| checkpoint_dir_name                        checkpoint_000000 |
| time_this_iter_s                                   288.47777 |
| time_total_s                                       288.47777 |
| training_iteration                                         1 |
| accuracy                                                0.91 |
| loss                                                 0.95535 |
| summary/epoch/0                                          1.0 |
| summary/train_acc/0                       0.5211970074812967 |
| summary/train_loss/0                      1.9375551560750375 |
| summary/val_acc/0                                       0.91 |
| summary/val_loss/0                 

In [ ]:
results.get_best_result(metric="accuracy", mode="max")

Result(
  metrics={'loss': 1.1160120623452323, 'accuracy': 0.91, 'summary': {'epoch': {0: 1.0}, 'train_loss': {0: 1.8922521781462889}, 'train_acc': {0: 0.5386533665835411}, 'val_loss': {0: 1.1160120623452323}, 'val_acc': {0: 0.91}}},
  path='/root/ray_results/TorchTrainer_2024-04-11_09-46-23/TorchTrainer_5f9ce_00001_1_batch_size=32,lr=0.0010_2024-04-11_09-46-31',
  filesystem='local',
  checkpoint=Checkpoint(filesystem=local, path=/root/ray_results/TorchTrainer_2024-04-11_09-46-23/TorchTrainer_5f9ce_00001_1_batch_size=32,lr=0.0010_2024-04-11_09-46-31/checkpoint_000000)
)

In [5]:
selected_model="resnet50"
root_base = os.path.join(project_folder, 'Incidents-subset')
root_augmented = os.path.join(project_folder, 'augmented_images')

data_folders=[root_base, root_augmented]

In [6]:
ciri_trainer = CIRI_trainer(model=selected_model,
                            data_folders=data_folders,
                            data_prop=0.8,
                            sample_indices=list(range(1000)), # remove this for full dataset
                            )

In [7]:
param_space = {
    "epochs": 1,
    "batch_size": tune.grid_search([32, 64])
}

In [ ]:
cv_results = ciri_trainer.cross_validate(
    run_name="test_cv",
    config=param_space,
    outer_cv_k=5,
    inner_cv_k=3,
    tune_hyperparameters=True,
    num_samples=5
)

In [7]:
cv_results = ciri_trainer.cross_validate(
    run_name="test_cv",
    config=None,
    outer_cv_k=5,
    inner_cv_k=0,
    tune_hyperparameters=False
)

Outer fold 0 - number of samples: 800


/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-04-12 09:01:31,662	INFO worker.py:1752 -- Started a local Ray instance.
2024-04-12 09:01:32,987	INFO tune.py:263 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `<FrameworkTrainer>(...)`.
2024-04-12 09:01:32,999	INFO tune.py:622 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949



View detailed results here: /root/ray_results/test_cv_outer_0
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-04-12_09-01-28_182196_347/artifacts/2024-04-12_09-01-33/test_cv_outer_0/driver_artifacts`


(TrainTrainable pid=1788) 2024-04-12 09:01:39.712221: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(TrainTrainable pid=1788) 2024-04-12 09:01:39.712283: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(TrainTrainable pid=1788) 2024-04-12 09:01:39.714023: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(TrainTrainable pid=1788) 2024-04-12 09:01:41.125186: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



Training started with configuration:
+---------------------------------------------------------+
| Training config                                         |
+---------------------------------------------------------+
| train_loop_config/data_folders     ...ugmented_images'] |
| train_loop_config/data_prop                         0.8 |
| train_loop_config/model                        resnet50 |
| train_loop_config/sample_indices   ...6, 997, 998, 999] |
| train_loop_config/train_test_idx   ..., 975, 994, 996]) |
+---------------------------------------------------------+


...


Training finished iteration 9 at 2024-04-12 09:21:41. Total running time: 4min 24s
+--------------------------------------------+
| Training result                            |
+--------------------------------------------+
| checkpoint_dir_name      checkpoint_000008 |
| time_this_iter_s                  27.51791 |
| time_total_s                      255.3398 |
| training_iteration                       9 |
| accuracy                             0.875 |
| loss                               2.40305 |
| summary/epoch/0                        1.0 |
| summary/epoch/1                        2.0 |
| summary/epoch/2                        3.0 |
| summary/epoch/3                        4.0 |
| summary/epoch/4                        5.0 |
| summary/epoch/5                        6.0 |
| summary/epoch/6                        7.0 |
| summary/epoch/7                        8.0 |
| summary/epoch/8                        9.0 |
| summary/train_acc/0     0.8057644110275689 |
| summary/train_acc/1  

(RayTrainWorker pid=7487) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/test_cv_outer_4/TorchTrainer_75258_00000_0_2024-04-12_09-17-17/checkpoint_000008)
Epoch (test) 10: 100%|██████████| 7/7 [00:04<00:00,  1.60it/s]



Training finished iteration 10 at 2024-04-12 09:22:08. Total running time: 4min 51s
+--------------------------------------------+
| Training result                            |
+--------------------------------------------+
| checkpoint_dir_name      checkpoint_000009 |
| time_this_iter_s                  27.11615 |
| time_total_s                     282.45595 |
| training_iteration                      10 |
| accuracy                             0.765 |
| loss                               0.51874 |
| summary/epoch/0                        1.0 |
| summary/epoch/1                        2.0 |
| summary/epoch/2                        3.0 |
| summary/epoch/3                        4.0 |
| summary/epoch/4                        5.0 |
| summary/epoch/5                        6.0 |
| summary/epoch/6                        7.0 |
| summary/epoch/7                        8.0 |
| summary/epoch/8                        9.0 |
| summary/epoch/9                       10.0 |
| summary/train_acc/0 

(RayTrainWorker pid=7487) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/test_cv_outer_4/TorchTrainer_75258_00000_0_2024-04-12_09-17-17/checkpoint_000009)
2024-04-12 09:22:09,971	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-04-12 09:22:09,977	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/test_cv_outer_4' in 0.0168s.



Training completed after 10 iterations at 2024-04-12 09:22:09. Total running time: 4min 52s

